In [7]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [8]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [9]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [10]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [11]:
dict_list = []
dummy = 0

for film_id in film_ids[0:1000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_1.json","w") as f:
      f.write(data)

with open("film_data_1.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Tom Selleck+_+Laura San Giacomo+_+Alan Rick...,"$20,000,000\n",+_+Australia+_+USA,+_+Simon Wincer,+_+Action+_+Adventure+_+Drama+_+Romance+_+Western,"$21,413,105\n",+_+long range rifle+_+australian aborigine+_+a...,+_+English+_+Aboriginal,PG-13,6.8,1990-10-19,119.0,Quigley Down Under (1990),"16,501",+_+John Hill,1990
1,+_+Kevin Costner+_+Mary McDonnell+_+Graham Gre...,"$22,000,000\n",+_+USA+_+UK,+_+Kevin Costner,+_+Adventure+_+Drama+_+Western,"$184,208,848\n",+_+friendship+_+soldier+_+19th century+_+honor...,+_+English+_+Sioux+_+Pawnee,PG-13,8.0,1990-11-21,181.0,Dances with Wolves (1990),"204,981",+_+Michael Blake+_+Michael Blake,1990
2,+_+Tony Todd+_+Patricia Tallman+_+Tom Towles+_...,"$4,200,000\n",+_+USA,+_+Tom Savini,+_+Horror,"$5,835,247\n",+_+biker+_+friends who live together+_+multipl...,+_+English,R,6.9,1990-10-19,92.0,Night of the Living Dead (1990),"33,738",+_+John A. Russo+_+George A. Romero,1990
3,+_+Susan Sarandon+_+James Spader+_+Jason Alexa...,None,+_+USA,+_+Luis Mandoki,+_+Drama+_+Romance,"$17,487,531\n",+_+older woman younger man relationship+_+love...,+_+English,R,6.5,1990-10-26,103.0,White Palace (1990),"6,480",+_+Glenn Savan+_+Ted Tally,1990
4,+_+Leo Fuchs+_+Eve Gordon+_+Lou Jacobi+_+Armin...,None,+_+USA,+_+Barry Levinson,+_+Drama,"$15,740,796\n",+_+trolley+_+suburb+_+baltimore maryland+_+nig...,+_+English+_+Yiddish,PG,7.3,1990-10-19,128.0,Avalon (1990),"5,115",+_+Barry Levinson,1990
5,+_+Jonathan Brandis+_+Kenny Morrison+_+Clariss...,"$36,000,000\n",+_+USA+_+Germany,+_+George Miller,+_+Adventure+_+Drama+_+Family+_+Fantasy,"$17,373,527\n",+_+book+_+reading+_+tower+_+empress+_+sorceress,+_+English,PG,5.1,1991-02-08,90.0,The NeverEnding Story II: The Next Chapter (1990),"20,152",+_+Karin Howard+_+Michael Ende,1990
6,+_+Ginger Lynn+_+Linnea Quigley+_+Jayne Hamil+...,None,+_+USA,+_+Rick Sloane,+_+Comedy+_+Crime,None,+_+arms tied overhead+_+bare chested male bond...,+_+English,R,4.3,1990-10-25,93.0,Vice Academy Part 2 (1990),231,+_+Rick Sloane,1990
7,+_+David Andrews+_+Kelly Wolf+_+Stephen Macht+...,"$10,500,000\n",+_+USA+_+Japan,+_+Ralph S. Singleton,+_+Horror,"$11,582,891\n",+_+textile mill+_+rat+_+water+_+boss subordina...,+_+English,R,4.8,1990-10-26,89.0,Graveyard Shift (1990),"7,824",+_+Stephen King+_+John Esposito,1990
8,+_+Kirstie Alley+_+Bill Pullman+_+Carrie Fishe...,None,+_+USA,+_+Carl Reiner,+_+Comedy,"$17,854,930\n",+_+police officer+_+shower+_+doctor+_+family r...,+_+English,PG-13,5.4,1990-10-26,87.0,Sibling Rivalry (1990),"2,141",+_+Martha Goldhirsh,1990
9,+_+Joe Estevez+_+Vivian Schilling+_+Gregg Thom...,"$242,000\n",+_+USA,+_+Michael Rissi,+_+Fantasy+_+Horror,"$233,150,",+_+hot bath+_+mother daughter relationship+_+s...,+_+English,R,2.3,1993-02-12,94.0,Soultaker (1990),"3,386",+_+Vivian Schilling+_+Eric Parkinson,1990
